## Tool Calling a.k.a Function Calling

In [61]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [49]:
# Python Decorators Refresher
"""
By definition, a decorator is a function that takes another function 
and extends the behavior of the latter function without explicitly modifying it
"""
import time

def timer(func):
    def wrapper(*args, **kwargs):
        starttime = time.perf_counter()
        print(f"Calling {func.__name__}")
        value = func(*args, **kwargs)
        endtime = time.perf_counter()
        time_taken = endtime - starttime
        print(f"{func.__name__} took {time_taken:4f} seconds to process")
        return value
    
    return wrapper

@timer
def say_hello(name,age):
    return f"Hello {name}, your age is {age}."

say_hello("Saurabh",10)

Calling say_hello
say_hello took 0.000033 seconds to process


'Hello Saurabh, your age is 10.'

### Using @tool decorator

While the name implies that the model is performing some action, this is actually <b><u>not</u></b> the case!
The model is coming up with the arguments to a tool, and actually running the tool.

Chat models supporting tool calling features implement a .bind_tools method, which receives a list of LangChain tool objects and binds them to the chat model in its expected format. Subsequent invocations of the chat model will include tool schemas in its calls to the LLM.

Tool Decorator makes tools out of functions

In [50]:
@tool
def multiply(a: int, b:int)->int:
    "Multiply a & b"
    return a*b

@tool
def add(a:int, b:int)->int:
    "Add a & b"
    return a+b

tools = [multiply,add]

### Or using pydantic

In [51]:
# This is just an example way of writing the code, these functions don't return anything
# In the below code, only above functions are used.
from langchain_core.pydantic_v1 import BaseModel, Field

class Add(BaseModel):
    """Add two integers together."""
    a:int=Field(...,description="First integer")
    b:int=Field(...,description="Second Integer")

class Multiply(BaseModel):
    """Multiply two integers together."""
    a:int=Field(...,description="First integer")
    b:int=Field(...,description="Second Integer")

tools = [Add, Multiply]

### LLM.bind_tools()
### llm_with_tools.invoke().tool_calls

In [53]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

#bind_tools
llm_with_tools = llm.bind_tools(tools)
print(llm_with_tools)

query = "What is 3*12? Also what is 3+3?"


# The role of LLM here is to just identify the right parameters and functions to call
# LLM is not resolving the user query using below line of code
result = llm_with_tools.invoke(query)
print(result)
print("**"*50)
print(result.tool_calls)



# To make a call to LLM and get result of a query, we have to call LLM.invoke
# This is the direct LLM call without the tools.
print("##"*50)
print(f"LLM Result\n{llm.invoke(query)}")

bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1318a8f50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x133a45370>, model_name='gpt-3.5-turbo-0125', openai_api_key=SecretStr('**********'), openai_proxy='') kwargs={'tools': [{'type': 'function', 'function': {'name': 'Add', 'description': 'Add two integers together.', 'parameters': {'type': 'object', 'properties': {'a': {'description': 'First integer', 'type': 'integer'}, 'b': {'description': 'Second Integer', 'type': 'integer'}}, 'required': ['a', 'b']}}}, {'type': 'function', 'function': {'name': 'Multiply', 'description': 'Multiply two integers together.', 'parameters': {'type': 'object', 'properties': {'a': {'description': 'First integer', 'type': 'integer'}, 'b': {'description': 'Second Integer', 'type': 'integer'}}, 'required': ['a', 'b']}}}]}
content='' additional_kwargs={'tool_calls': [{'id': 'call_ZNrCBkNKA0gtpiSDakHFoUw4', 'function': {'arguments': '{"a": 3, "b

### tool_calls is the main object that performs the major job

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

#bind_tools
llm_with_tools = llm.bind_tools(tools)
print(llm_with_tools)

query = "What is 3*12? Also what is 3+3?"


# The role of LLM here is to just identify the right parameters and functions to call
# LLM is not resolving the user query using below line of code
result = llm_with_tools.invoke(query)
print(result)
print("**"*50)
print(result.tool_calls)



# To make a call to LLM and get result of a query, we have to call LLM.invoke
# This is the direct LLM call without the tools.
print("##"*50)
print(f"LLM Result\n{llm.invoke(query)}")

bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1318a8f50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x133a45370>, model_name='gpt-3.5-turbo-0125', openai_api_key=SecretStr('**********'), openai_proxy='') kwargs={'tools': [{'type': 'function', 'function': {'name': 'Add', 'description': 'Add two integers together.', 'parameters': {'type': 'object', 'properties': {'a': {'description': 'First integer', 'type': 'integer'}, 'b': {'description': 'Second Integer', 'type': 'integer'}}, 'required': ['a', 'b']}}}, {'type': 'function', 'function': {'name': 'Multiply', 'description': 'Multiply two integers together.', 'parameters': {'type': 'object', 'properties': {'a': {'description': 'First integer', 'type': 'integer'}, 'b': {'description': 'Second Integer', 'type': 'integer'}}, 'required': ['a', 'b']}}}]}
content='' additional_kwargs={'tool_calls': [{'id': 'call_ZNrCBkNKA0gtpiSDakHFoUw4', 'function': {'arguments': '{"a": 3, "b

In [36]:
# Using Output Parser to format the output
# Convert the output back to original pydantic class

# PydanticToolsParser needs a pydantic base class type of tools list

from langchain_core.output_parsers.openai_tools import PydanticToolsParser

output_parser=PydanticToolsParser(tools=[Multiply, Add])

chain = llm_with_tools | output_parser

chain.invoke(query)

[Multiply(a=3, b=12), Add(a=3, b=3)]

### Passing tool outputs to model
Using ToolMessage

In [41]:
from langchain_core.messages import ToolMessage, HumanMessage

messages = [HumanMessage(content=query)]
ai_message = llm_with_tools.invoke(query)
messages.append(ai_message)

for tool_call in ai_message.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    print(f"printing selected tool {selected_tool}")
    #It's on user when they want to invoke the tool
    tool_output = selected_tool.invoke(tool_call["args"])
    print(f"printing tool output {tool_output}")
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
messages

printing selected tool name='multiply' description='multiply(a: int, b: int) -> int - Multiply a & b' args_schema=<class 'pydantic.v1.main.multiplySchema'> func=<function multiply at 0x125131bc0>
printing tool output 36
printing selected tool name='add' description='add(a: int, b: int) -> int - Add a & bs' args_schema=<class 'pydantic.v1.main.addSchema'> func=<function add at 0x1251318a0>
printing tool output 6


[HumanMessage(content='What is 3*12? Also what is 3+3?'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WNGIvts2PNivb95FKu9kwKCb', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'Multiply'}, 'type': 'function'}, {'id': 'call_0ONTAtbTj52W5oUDHJp17vpd', 'function': {'arguments': '{"a": 3, "b": 3}', 'name': 'Add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 90, 'total_tokens': 140}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_caf95bb1ae', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a8185c01-48da-4cf9-9290-a7127ac59648-0', tool_calls=[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_WNGIvts2PNivb95FKu9kwKCb'}, {'name': 'Add', 'args': {'a': 3, 'b': 3}, 'id': 'call_0ONTAtbTj52W5oUDHJp17vpd'}]),
 ToolMessage(content='36', tool_call_id='call_WNGIvts2PNivb95FKu9kwKCb'),
 ToolMessage(content='6', tool_call_id='call_0ONTAtbTj52W5oUDHJp17vpd')]

## Few Shot Prompting

In [54]:
query = "Whats 119 times 8 minus 20. Don't do any math yourself, only use tools for math. Respect order of operations"

llm_with_tools.invoke(query).tool_calls

[{'name': 'Multiply',
  'args': {'a': 119, 'b': 8},
  'id': 'call_EjJGiBt5QpJYyNmG3RNzCt36'},
 {'name': 'Add',
  'args': {'a': 952, 'b': -20},
  'id': 'call_PD5SuJLkNIUYvYREKwtxUj0r'}]

#### In the above result, it shouldn't call the add method since we have not invoked the multiply method yet.
#### This problem can be solved with Few Shot Prompting

In [69]:
examples = [
    HumanMessage("What's the product of 317253 and 128472 plus four", name="example_user"),
    AIMessage(content="", 
              name="example_assistant",
              tool_calls=[{
                  "name":"Multiply",
                  "args":{'a':317253, 'b':128472},
                  "id":"1"
              }]),
    ToolMessage("40758127416", tool_call_id="1"),
    AIMessage(content="",
              name = "example_assistant",
              tool_calls=[{
                  "name":"Add",
                  "args":{'a':40758127416, 'b':4},
                  "id":"2"
              }]
              ),
    ToolMessage("40758127420", tool_call_id="2"),
    AIMessage(content="The product of 317253 and 128472 plus 4 is 40758127420")
]


system = """You are really bad at math but good at using calculator.
Use below tool usage as an example of how to correctly use the tools
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system",system),
    *examples,
    ("human","{query}")
])

chain = {"query":RunnablePassthrough()}|few_shot_prompt|llm_with_tools
chain.invoke("Whats 119 times 8 minus 20").tool_calls

[{'name': 'Multiply',
  'args': {'a': 119, 'b': 8},
  'id': 'call_eUpUCx6PNzjaGlK2ADcfu90W'}]